In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import implicit
import scipy.sparse as scs

from recsys.utils import col
from recsys.metrics import ndcg_score, hr_score

### Last one

In [ ]:
X = pd.read_parquet("../data/ml-1m/split/X_last_one.parquet")
y = pd.read_parquet("../data/ml-1m/split/y_last_one.parquet")

In [ ]:
y_true = y[[col.movie_code]].to_numpy()

test_array = np.hstack([y_true, np.array(y[col.negative].apply(list).tolist())])

user_movie_matrix = scs.csr_matrix(
    (X[col.rating], (X[col.user_code], X[col.movie_code]))
)

In [ ]:
model = implicit.als.AlternatingLeastSquares(
    factors=128,
    iterations=100,
    use_gpu=True,
    calculate_training_loss=True
)

model.fit(user_movie_matrix)

In [ ]:
y_pred = []
for uid in tqdm(np.arange(user_movie_matrix.shape[0])):
    pred, score = model.recommend(
        uid,
        user_items=user_movie_matrix[uid],
        items=test_array[uid],
    )
    y_pred.append(pred)
    
y_pred = np.array(y_pred)

In [ ]:
ndcg_score(y_true, y_pred)

In [ ]:
hr_score(y_true, y_pred)

### Last five

In [ ]:
X = pd.read_parquet("../data/ml-1m/split/X_last_five.parquet")
y = pd.read_parquet("../data/ml-1m/split/y_last_five.parquet")

In [ ]:
most_popular = X[col.movie_code].value_counts().index.tolist()
user_ids = X[col.user_code].unique()
y_true = np.array(
    y.groupby(col.user_code)[col.movie_code].unique().apply(list).tolist()
)

In [ ]:
user_movie_matrix = scs.coo_matrix(
    (X[col.rating], (X[col.user_code], X[col.movie_code]))
).tocsr()

In [ ]:
embedding_dim = 128

model = implicit.als.AlternatingLeastSquares(
    factors=embedding_dim,
    iterations=50,
)

model.fit(user_movie_matrix)

In [ ]:
y_pred, y_score = model.recommend(
    X[col.user_code].unique(), user_items=user_movie_matrix
)

In [ ]:
ndcg_score(y_true, y_pred)